<a href="https://colab.research.google.com/github/Saranyanv/test/blob/master/test_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [0]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size 
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [0]:
from IPython.display import clear_output, Image, display, HTML
import datetime

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def
  
  
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [0]:
import numpy as np
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [0]:
n_inputs = 28*28  # MNIST
n_hidden1_1 = 300
n_hidden1_2 = 100
n_hidden2_1 = 200
n_hidden2_2 = 100
n_outputs = 10

In [0]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [0]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [0]:
with tf.name_scope("dnn1"):
    hidden1 = tf.layers.dense(X, n_hidden1_1, name="hidden1",
                              activation=tf.nn.softmax)
    hidden2 = tf.layers.dense(hidden1, n_hidden1_2, name="hidden2",
                              activation=tf.nn.softmax)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)

In [0]:
with tf.name_scope("loss1"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [0]:
learning_rate = 0.01

with tf.name_scope("train1"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [0]:
with tf.name_scope("eval1"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [355]:
n_epochs = 20
batch_size = 50

start_tm=datetime.datetime.now()
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)
end_tm=datetime.datetime.now()
print ("Time taken to fit is ",end_tm - start_tm)

0 Batch accuracy: 0.16 Validation accuracy: 0.1126
1 Batch accuracy: 0.12 Validation accuracy: 0.1126
2 Batch accuracy: 0.06 Validation accuracy: 0.1126
3 Batch accuracy: 0.14 Validation accuracy: 0.1126
4 Batch accuracy: 0.14 Validation accuracy: 0.1126
5 Batch accuracy: 0.08 Validation accuracy: 0.1126
6 Batch accuracy: 0.14 Validation accuracy: 0.1126
7 Batch accuracy: 0.14 Validation accuracy: 0.1126
8 Batch accuracy: 0.18 Validation accuracy: 0.1126
9 Batch accuracy: 0.1 Validation accuracy: 0.1126
10 Batch accuracy: 0.12 Validation accuracy: 0.1126
11 Batch accuracy: 0.14 Validation accuracy: 0.1126
12 Batch accuracy: 0.12 Validation accuracy: 0.1126
13 Batch accuracy: 0.06 Validation accuracy: 0.1126
14 Batch accuracy: 0.06 Validation accuracy: 0.1126
15 Batch accuracy: 0.14 Validation accuracy: 0.1126
16 Batch accuracy: 0.08 Validation accuracy: 0.1126
17 Batch accuracy: 0.06 Validation accuracy: 0.1126
18 Batch accuracy: 0.12 Validation accuracy: 0.1126
19 Batch accuracy: 0.14

In [356]:
with tf.Session() as sess:
    init.run()
    Z_train = logits.eval(feed_dict={X: X_train})
    y_train_pred = np.argmax(Z_train, axis=1)
    
    Z_test = logits.eval(feed_dict={X: X_test})
    y_test_pred = np.argmax(Z_test, axis=1)
    
    accuracy_train = accuracy.eval(feed_dict={X: X_train, y: y_train})
    accuracy_test = accuracy.eval(feed_dict={X: X_test, y: y_test})  
    
    print ("Accuracy on Training set = ",accuracy_train)
    print ("Accuracy on Testing set = ",accuracy_test)

    
#Cross-verify with Sklearn accuracy scores    
#from sklearn.metrics import accuracy_score
#a=accuracy_score(y_train_pred, y_train)
#b=accuracy_score(y_test_pred, y_test)
#print (a,b)

Accuracy on Training set =  0.11234546
Accuracy on Testing set =  0.1135


In [357]:
#show_graph(tf.get_default_graph())
show_graph(tf.get_default_graph().as_graph_def())

In [0]:
reset_graph()
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [0]:
with tf.name_scope("dnn2"):
    hidden1 = tf.layers.dense(X, n_hidden1_1, name="hidden1",
                              activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden1_2, name="hidden2",
                              activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)

In [0]:
with tf.name_scope("loss2"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")   

In [0]:
learning_rate = 0.01

with tf.name_scope("train2"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [0]:
with tf.name_scope("eval2"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [364]:
n_epochs = 20
batch_size = 50

start_tm=datetime.datetime.now()
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)
end_tm=datetime.datetime.now()
print ("Time taken to fit is ",end_tm - start_tm)

0 Batch accuracy: 0.9 Validation accuracy: 0.9024
1 Batch accuracy: 0.92 Validation accuracy: 0.9254
2 Batch accuracy: 0.94 Validation accuracy: 0.9372
3 Batch accuracy: 0.9 Validation accuracy: 0.9416
4 Batch accuracy: 0.94 Validation accuracy: 0.947
5 Batch accuracy: 0.94 Validation accuracy: 0.9512
6 Batch accuracy: 1.0 Validation accuracy: 0.9548
7 Batch accuracy: 0.94 Validation accuracy: 0.961
8 Batch accuracy: 0.96 Validation accuracy: 0.9622
9 Batch accuracy: 0.94 Validation accuracy: 0.9648
10 Batch accuracy: 0.92 Validation accuracy: 0.9656
11 Batch accuracy: 0.98 Validation accuracy: 0.9668
12 Batch accuracy: 0.98 Validation accuracy: 0.9684
13 Batch accuracy: 0.98 Validation accuracy: 0.9704
14 Batch accuracy: 1.0 Validation accuracy: 0.9698
15 Batch accuracy: 0.94 Validation accuracy: 0.9718
16 Batch accuracy: 0.98 Validation accuracy: 0.9728
17 Batch accuracy: 1.0 Validation accuracy: 0.973
18 Batch accuracy: 0.98 Validation accuracy: 0.9746
19 Batch accuracy: 0.98 Valida

In [365]:
with tf.Session() as sess:
    init.run()
    Z_train = logits.eval(feed_dict={X: X_train})
    y_train_pred = np.argmax(Z_train, axis=1)
    
    Z_test = logits.eval(feed_dict={X: X_test})
    y_test_pred = np.argmax(Z_test, axis=1)
    
    accuracy_train = accuracy.eval(feed_dict={X: X_train, y: y_train})
    accuracy_test = accuracy.eval(feed_dict={X: X_test, y: y_test})  
    
    print ("Accuracy on Training set = ",accuracy_train)
    print ("Accuracy on Testing set = ",accuracy_test)

    
#Cross-verify with Sklearn accuracy scores    
#from sklearn.metrics import accuracy_score
#a=accuracy_score(y_train_pred, y_train)
#b=accuracy_score(y_test_pred, y_test)
#print (a,b)

Accuracy on Training set =  0.069290906
Accuracy on Testing set =  0.0654


In [366]:
show_graph(tf.get_default_graph().as_graph_def())

In [0]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [0]:
with tf.name_scope("dnn3"):
    hidden1 = tf.layers.dense(X, n_hidden2_1, name="hidden1",
                              activation=tf.nn.softmax)
    hidden2 = tf.layers.dense(hidden1, n_hidden2_2, name="hidden2",
                              activation=tf.nn.softmax)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)

In [0]:
with tf.name_scope("loss3"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss") 

In [0]:
learning_rate = 0.01

with tf.name_scope("train3"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [0]:
with tf.name_scope("eval3"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [373]:
n_epochs = 20
batch_size = 50

start_tm=datetime.datetime.now()
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)
end_tm=datetime.datetime.now()
print ("Time taken to fit is ",end_tm - start_tm)

0 Batch accuracy: 0.16 Validation accuracy: 0.1126
1 Batch accuracy: 0.12 Validation accuracy: 0.1126
2 Batch accuracy: 0.06 Validation accuracy: 0.1126
3 Batch accuracy: 0.14 Validation accuracy: 0.1126
4 Batch accuracy: 0.14 Validation accuracy: 0.1126
5 Batch accuracy: 0.08 Validation accuracy: 0.1126
6 Batch accuracy: 0.14 Validation accuracy: 0.1126
7 Batch accuracy: 0.14 Validation accuracy: 0.1126
8 Batch accuracy: 0.18 Validation accuracy: 0.1126
9 Batch accuracy: 0.1 Validation accuracy: 0.1126
10 Batch accuracy: 0.12 Validation accuracy: 0.1126
11 Batch accuracy: 0.14 Validation accuracy: 0.1126
12 Batch accuracy: 0.12 Validation accuracy: 0.1126
13 Batch accuracy: 0.06 Validation accuracy: 0.1126
14 Batch accuracy: 0.06 Validation accuracy: 0.1126
15 Batch accuracy: 0.14 Validation accuracy: 0.1126
16 Batch accuracy: 0.08 Validation accuracy: 0.1126
17 Batch accuracy: 0.06 Validation accuracy: 0.1126
18 Batch accuracy: 0.12 Validation accuracy: 0.1126
19 Batch accuracy: 0.14

In [374]:
with tf.Session() as sess:
    init.run()
    Z_train = logits.eval(feed_dict={X: X_train})
    y_train_pred = np.argmax(Z_train, axis=1)
    
    Z_test = logits.eval(feed_dict={X: X_test})
    y_test_pred = np.argmax(Z_test, axis=1)
    
    accuracy_train = accuracy.eval(feed_dict={X: X_train, y: y_train})
    accuracy_test = accuracy.eval(feed_dict={X: X_test, y: y_test})  
    
    print ("Accuracy on Training set = ",accuracy_train)
    print ("Accuracy on Testing set = ",accuracy_test)

    
#Cross-verify with Sklearn accuracy scores    
#from sklearn.metrics import accuracy_score
#a=accuracy_score(y_train_pred, y_train)
#b=accuracy_score(y_test_pred, y_test)
#print (a,b)

Accuracy on Training set =  0.11234546
Accuracy on Testing set =  0.1135


In [375]:
show_graph(tf.get_default_graph().as_graph_def())

In [0]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [0]:
with tf.name_scope("dnn4"):
    hidden1 = tf.layers.dense(X, n_hidden2_1, name="hidden1",
                              activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2_2, name="hidden2",
                              activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)

In [0]:
with tf.name_scope("loss4"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss") 

In [0]:
learning_rate = 0.01

with tf.name_scope("train4"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [0]:
with tf.name_scope("eval4"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [410]:
n_epochs = 12
batch_size = 50
final_model_path = "./model4"

start_tm=datetime.datetime.now()
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        saver.save(sess, final_model_path)
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        #acc_valid = accuracy.eval(feed_dict={X: X_train, y: y_train})
        print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)
end_tm=datetime.datetime.now()
print ("Time taken to fit is ",end_tm - start_tm)

0 Batch accuracy: 0.88 Validation accuracy: 0.9028
1 Batch accuracy: 0.92 Validation accuracy: 0.9226
2 Batch accuracy: 0.88 Validation accuracy: 0.9314
3 Batch accuracy: 1.0 Validation accuracy: 0.9394
4 Batch accuracy: 0.96 Validation accuracy: 0.944
5 Batch accuracy: 0.96 Validation accuracy: 0.9482
6 Batch accuracy: 0.94 Validation accuracy: 0.952
7 Batch accuracy: 0.98 Validation accuracy: 0.9554
8 Batch accuracy: 0.94 Validation accuracy: 0.9576
9 Batch accuracy: 0.96 Validation accuracy: 0.9598
10 Batch accuracy: 0.98 Validation accuracy: 0.9636
11 Batch accuracy: 1.0 Validation accuracy: 0.9646
Time taken to fit is  0:00:27.410623


In [413]:
with tf.Session() as sess:
    saver.restore(sess, final_model_path)
    train_Accuracy_val = accuracy.eval(feed_dict={X: X_train, y: y_train})
    test_Accuracy_val = accuracy.eval(feed_dict={X: X_test, y: y_test})

INFO:tensorflow:Restoring parameters from ./my_deep_mnist_model


In [414]:
print ("Training Accuracy=",train_Accuracy_val)
print ("Testing Accuracy=",test_Accuracy_val)

Training Accuracy= 0.9635818
Training Accuracy= 0.9597


In [403]:
n_epochs = 12
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict={X: X_train, y: y_train})
        acc_valid = accuracy.eval(feed_dict={X: X_train, y: y_train})
        print(epoch, "Training accuracy:", acc_valid)

0 Training accuracy: 0.11309091
1 Training accuracy: 0.11436363
2 Training accuracy: 0.11641818
3 Training accuracy: 0.11803637
4 Training accuracy: 0.12078182
5 Training accuracy: 0.124145456
6 Training accuracy: 0.12770909
7 Training accuracy: 0.1316909
8 Training accuracy: 0.13678181
9 Training accuracy: 0.14241818
10 Training accuracy: 0.1481091
11 Training accuracy: 0.15409091


In [409]:
n_epochs = 12
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict={X: X_test, y: y_test})
        acc_valid = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Training accuracy:", acc_valid)

0 Training accuracy: 0.1126
1 Training accuracy: 0.114
2 Training accuracy: 0.1152
3 Training accuracy: 0.118
4 Training accuracy: 0.1207
5 Training accuracy: 0.1239
6 Training accuracy: 0.1282
7 Training accuracy: 0.132
8 Training accuracy: 0.1375
9 Training accuracy: 0.1429
10 Training accuracy: 0.1501
11 Training accuracy: 0.1554


In [384]:
show_graph(tf.get_default_graph().as_graph_def())